Wybrałam modele z publikacji załączonej jako pdf: shallow (surowy sygnał EEG) i Reimann (wyodrębnione cechy sygnału EEG). Zestaw danych użyty w tej publikacji to sygnały EEG (https://isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_abnormal/v2.0.0/) które mają w sobie informacje o wieku, płci pacjenta oraz czy pacjent był zdrowy czy chory. Nie wgrywałam ich ze względu na duży rozmiar. Udostępnili również githuba ze swoimi kodami: https://github.com/gemeinl/auto-eeg-diagnosis-comparison

Ponieważ bardzo mnie to interesuje to się uparłam żeby to przygotować jako pracę domową, jednak okazało się że tydzień to za mało czasu, bo jeszcze nie do końca się odnalazłam w ich githubie - jest bardzo nieintuicyjny niestety. 

Tak więc bardzo przepraszam za ubogą pracę domową. Do następnych zajęć postaram się stworzyć własny moduł nad którym będę mogła dalej pracować dodając metody z XAI. 

To piszę tak bardziej informacyjnie, bo pamiętam że wspominał Pan że na podstawie tych prac domowych będą tworzone grupy.